In [ ]:
# Dependencies
import requests

import time
from dotenv import load_dotenv
import os
import pandas as pd
import json


from datetime import datetime
def convert_to_date(time_period_str):
    """Converts a time period string (YYYY-MM) to a datetime object."""

    return datetime.strptime(time_period_str, "%Y-%m")

## Load the FRED_API_KEY and BEA_API_KEY from the env file
load_dotenv()
FRED_API_KEY = os.getenv('FRED_API_KEY')
BEA_API_KEY = os.getenv('BEA_API_KEY')


APIs: Federal Reserve Economic Data (FRED) and Bureau of Economic Analysis (BEA)

In [14]:
# Set the base URL to NASA's DONKI API:
fred_base_url = "https://api.stlouisfed.org/fred/series/observations?"
series_id_date="series_id=FEDFUNDS&realtime_start=1996-12-03&realtime_end=2024-12-02&"


# Build URL for FRED
fred_query_url=fred_base_url+series_id_date+"api_key="+FRED_API_KEY+"&file_type=json"



In [15]:
bea_query_url = "https://apps.bea.gov/api/data/?&UserID="+BEA_API_KEY+"&method=GetData&DataSetName=NIPA&TableName=T20807&Frequency=M&Year=ALL&ResultFormat=json"


In [16]:
fred_response_data = requests.get(fred_query_url)
fred_data = fred_response_data.json()
print(json.dumps(fred_data, indent=4))

#real time start and end 1996-12-03 and 2024-12-02


{
    "realtime_start": "1996-12-03",
    "realtime_end": "2024-12-02",
    "observation_start": "1600-01-01",
    "observation_end": "9999-12-31",
    "units": "lin",
    "output_type": 1,
    "file_type": "json",
    "order_by": "observation_date",
    "sort_order": "asc",
    "count": 899,
    "offset": 0,
    "limit": 100000,
    "observations": [
        {
            "realtime_start": "1996-12-03",
            "realtime_end": "2024-12-02",
            "date": "1954-07-01",
            "value": "0.80"
        },
        {
            "realtime_start": "1996-12-03",
            "realtime_end": "2024-12-02",
            "date": "1954-08-01",
            "value": "1.22"
        },
        {
            "realtime_start": "1996-12-03",
            "realtime_end": "2020-07-20",
            "date": "1954-09-01",
            "value": "1.06"
        },
        {
            "realtime_start": "2020-07-21",
            "realtime_end": "2020-08-04",
            "date": "1954-09-01",
         

In [8]:
bea_response_data = requests.get(bea_query_url)
bea_data = bea_response_data.json()
#print(json.dumps(bea_data, indent=4))

In [17]:
#In order to read and plot observations of data,
#one must get the date and value nested inside of the json dictionary
fred_df = pd.json_normalize(fred_data,'observations')

fred_df.head(10)

,realtime_start,realtime_end,date,value
0,1996-12-03,2024-12-02,1954-07-01,0.80
1,1996-12-03,2024-12-02,1954-08-01,1.22
2,1996-12-03,2020-07-20,1954-09-01,1.06
3,2020-07-21,2020-08-04,1954-09-01,1.07
4,2020-08-05,2020-10-12,1954-09-01,1.06
5,2020-10-13,2024-12-02,1954-09-01,1.07
6,1996-12-03,2024-12-02,1954-10-01,0.85
7,1996-12-03,2024-12-02,1954-11-01,0.83
8,1996-12-03,2024-12-02,1954-12-01,1.28
9,1996-12-03,2024-12-02,1955-01-01,1.39


In [ ]:
#In order to read and plot observations of data,
#one must get the date and value nested inside of the json dictionary to the third level
bea_df = pd.json_normalize(bea_data["BEAAPI"]["Results"]["Data"])

bea_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23787 entries, 0 to 23786
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   TableName        23787 non-null  object
 1   SeriesCode       23787 non-null  object
 2   LineNumber       23787 non-null  object
 3   LineDescription  23787 non-null  object
 4   TimePeriod       23787 non-null  object
 5   METRIC_NAME      23787 non-null  object
 6   CL_UNIT          23787 non-null  object
 7   UNIT_MULT        23787 non-null  object
 8   DataValue        23787 non-null  object
 9   NoteRef          23787 non-null  object
dtypes: object(10)
memory usage: 1.8+ MB


In [42]:
bea_test=bea_df['TimePeriod'].str.split('M')

bea_dates= bea_test.apply(lambda x: pd.to_datetime(f"{x[0]}-{x[1]}"))
bea_dates

0       1959-02-01
1       1959-03-01
2       1959-04-01
3       1959-05-01
4       1959-06-01
           ...    
23782   2024-06-01
23783   2024-07-01
23784   2024-08-01
23785   2024-09-01
23786   2024-10-01
Name: TimePeriod, Length: 23787, dtype: datetime64[ns]

In [43]:
bea_time_updated=bea_df.copy()
bea_time_updated["Time Updated"]=bea_dates


In [53]:
pce_df = bea_time_updated.loc[bea_time_updated['LineDescription'] == 'Personal consumption expenditures (PCE)']

In [54]:
pce_df

,TableName,SeriesCode,LineNumber,LineDescription,TimePeriod,METRIC_NAME,CL_UNIT,UNIT_MULT,DataValue,NoteRef,Time Updated
0,T20807,DPCERGM,1,Personal consumption expenditures (PCE),1959M02,Fisher Price Index,Percent change,0,0.1,T20807,1959-02-01
1,T20807,DPCERGM,1,Personal consumption expenditures (PCE),1959M03,Fisher Price Index,Percent change,0,0.1,T20807,1959-03-01
2,T20807,DPCERGM,1,Personal consumption expenditures (PCE),1959M04,Fisher Price Index,Percent change,0,0.2,T20807,1959-04-01
3,T20807,DPCERGM,1,Personal consumption expenditures (PCE),1959M05,Fisher Price Index,Percent change,0,0.1,T20807,1959-05-01
4,T20807,DPCERGM,1,Personal consumption expenditures (PCE),1959M06,Fisher Price Index,Percent change,0,0.3,T20807,1959-06-01
...,...,...,...,...,...,...,...,...,...,...,...
784,T20807,DPCERGM,1,Personal consumption expenditures (PCE),2024M06,Fisher Price Index,Percent change,0,0.1,T20807,2024-06-01
785,T20807,DPCERGM,1,Personal consumption expenditures (PCE),2024M07,Fisher Price Index,Percent change,0,0.2,T20807,2024-07-01
786,T20807,DPCERGM,1,Personal consumption expenditures (PCE),2024M08,Fisher Price Index,Percent change,0,0.1,T20807,2024-08-01
787,T20807,DPCERGM,1,Personal consumption expenditures (PCE),2024M09,Fisher Price Index,Percent change,0,0.2,T20807,2024-09-01
